In [1]:
import os

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [3]:
import kerastuner
kerastuner.__version__

'1.0.2'

In [4]:
import glob
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.imagenet_utils import preprocess_input
from IPython.display import SVG
from tensorflow.keras.utils import plot_model
#from resnets_utils import *
from tensorflow.keras.initializers import he_normal
import scipy.misc
from matplotlib.pyplot import imshow
%matplotlib inline

from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard

from tensorflow.keras import backend as K
K.set_image_data_format('channels_last')
K.set_learning_phase(1)

from kerastuner.tuners import RandomSearch, Hyperband
from kerastuner.engine.hyperparameters import HyperParameters

/home/8p361-lecturer/anaconda3/lib/python3.8/site-packages/tensorflow/python/keras/backend.py:434: UserWarning: `tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
  warnings.warn('`tf.keras.backend.set_learning_phase` is deprecated and '


In [5]:
def identity_block(X, f, filters, stage, block): 
        
    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    # Retrieve Filters
    F1, F2, F3 = filters
    
    # Save the input value. You'll need this later to add back to the main path. 
    X_shortcut = X
    
    # First component of main path
    X = Conv2D(filters = F1, kernel_size = (f, f), strides = (1,1), padding = 'same', name = conv_name_base + '2a', kernel_initializer = he_normal(seed=None))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = Activation('relu')(X)
    
    # Second component of main path 
    X = Conv2D(filters = F2, kernel_size = (f, f), strides = (1,1), padding = 'same', name = conv_name_base + '2b', kernel_initializer = he_normal(seed=None))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2b')(X)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation 
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)
    
    
    return X

In [6]:
def convolutional_block(X, f, filters, stage, block, s = 2):
        
    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    # Retrieve Filters
    F1, F2, F3 = filters
    
    # Save the input value
    X_shortcut = X

    ##### MAIN PATH #####
    # First component of main path 
    X = Conv2D(filters = F1,kernel_size = (f, f), strides = (s,s), name = conv_name_base + '2a', kernel_initializer = he_normal(seed=None))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = Activation('relu')(X)

    # Second component of main path 
    X = Conv2D(filters = F2, kernel_size = (f, f), strides = (1,1), padding = 'same', name = conv_name_base + '2b', kernel_initializer = he_normal(seed=None))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2b')(X)

    ##### SHORTCUT PATH #### 
    X_shortcut = Conv2D(filters = F2, kernel_size = (3, 3), strides = (s,s), padding = 'valid', name = conv_name_base + '1',
                        kernel_initializer = he_normal(seed=None))(X_shortcut)
    X_shortcut = BatchNormalization(axis = 3, name = bn_name_base + '1')(X_shortcut)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation 
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)
    
    
    return X
    

In [7]:
def ResNet(input_shape=(96, 96, 3), classes=1):
    """
    Implementation of the popular ResNet50 the following architecture:
    CONV2D -> BATCHNORM -> RELU -> MAXPOOL -> CONVBLOCK -> IDBLOCK*2 -> CONVBLOCK -> IDBLOCK*3
    -> CONVBLOCK -> IDBLOCK*5 -> CONVBLOCK -> IDBLOCK*2 -> AVGPOOL -> TOPLAYER

    Arguments:
    input_shape -- shape of the images of the dataset
    classes -- integer, number of classes

    Returns:
    model -- a Model() instance in Keras
    """

    # Define the input as a tensor with shape input_shape
    X_input = Input(input_shape)

    # Zero-Padding
    X = ZeroPadding2D((3, 3))(X_input)

    # Stage 1
    X = Conv2D(64, (7, 7), strides=(2, 2), name='conv1', kernel_initializer=he_normal(seed=None))(X)
    X = BatchNormalization(axis=3, name='bn_conv1')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)

    # Stage 2
    X = convolutional_block(X, f=3, filters=[64, 64, 256], stage=2, block='a', s=1)
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='m')
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='n')

    # Stage 3
    X = convolutional_block(X, f=3, filters=[128,128,512], stage=3, block='a', s=2)
    X = identity_block(X, 3, [128,128,512], stage=3, block='b')
    X = identity_block(X, 3, [128,128,512], stage=3, block='c')
    X = identity_block(X, 3, [128,128,512], stage=3, block='d')
    X = identity_block(X, 3, [128,128,512], stage=3, block='e')
    X = identity_block(X, 3, [128,128,512], stage=3, block='f')
    X = identity_block(X, 3, [128,128,512], stage=3, block='g')
    X = identity_block(X, 3, [128,128,512], stage=3, block='h')
    X = identity_block(X, 3, [128,128,512], stage=3, block='i')
    X = identity_block(X, 3, [128,128,512], stage=3, block='j')
    X = identity_block(X, 3, [128,128,512], stage=3, block='k')
    X = identity_block(X, 3, [128,128,512], stage=3, block='l')

    # Stage 4
    X = convolutional_block(X, f=3, filters=[256,256,1024], stage=4, block='a', s=2)
    X = identity_block(X, 3, [256,256,1024], stage=4, block='o')
    X = identity_block(X, 3, [256,256,1024], stage=4, block='p')
    
    # Stage 5
    X = convolutional_block(X, f=3, filters=[512,512,2048], stage=5, block='a', s=2)
    X = identity_block(X, 3, [512,512,2048], stage=5, block='q')
    X = identity_block(X, 3, [512,512,2048], stage=5, block='r') 
    

    ### END CODE HERE ###

    # output layer
    X = Flatten()(X)
    X = Dense(classes, activation='sigmoid', name='fc' + str(classes), kernel_initializer = he_normal(seed=None))(X)
    
    
    # Create model
    model = Model(inputs = X_input, outputs = X)
    
    model.compile(SGD(lr=1e-6, momentum=0.95), loss = 'binary_crossentropy', metrics=['accuracy'])

    return model

In [8]:
#model.compile(SGD(lr=0.001, momentum=0.95), loss='binary_crossentropy', metrics=['accuracy'])

In [9]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

########### insert hyperparameters ################
train_batch_size = 32
val_batch_size = 32
###################################################
base_dir = r'data'
test_dir = r'data\test'

# dataset parameters
TRAIN_PATH = os.path.join(base_dir, 'train')
VALID_PATH = os.path.join(base_dir, 'valid')
TEST_FILES = glob.glob(test_dir + '\*.tif')
RESCALING_FACTOR = 1./255
IMAGE_SIZE = 96

# instantiate data generators
datagen = ImageDataGenerator(rescale=RESCALING_FACTOR)

train_gen = datagen.flow_from_directory(TRAIN_PATH,
                                    target_size=(IMAGE_SIZE, IMAGE_SIZE),
                                    batch_size=train_batch_size,
                                    class_mode='binary',
                                    shuffle=True)

val_gen = datagen.flow_from_directory(VALID_PATH,
                                    target_size=(IMAGE_SIZE, IMAGE_SIZE),
                                    batch_size=val_batch_size,
                                    class_mode='binary',
                                    shuffle=False)

# form steps
train_steps = train_gen.n//train_gen.batch_size
val_steps = val_gen.n//val_gen.batch_size

Found 144000 images belonging to 2 classes.
Found 16000 images belonging to 2 classes.


In [10]:
#tuner = Hyperband(
#    build_model,
#    overwrite=True,
#    objective='val_accuracy',
#    max_epochs = 15,
#    factor = 2,
#    hyperband_iterations=1,
#    directory ='storage/ResnetTuner_s',
#    project_name='ResNet_Stage1234_s'
#)

In [11]:
#stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

In [12]:
#tuner.search(train_gen,
#             verbose=2, 
#             epochs=25,
#             batch_size=32,
#             steps_per_epoch=train_steps,
#             callbacks=[stop_early],
#             validation_steps=val_steps,
#             validation_data=val_gen)

In [13]:
model = ResNet()

In [14]:
# save the model and weights
model_name = 'ResNet_Stage12345_s'
model_filepath = model_name + '.json'
weights_filepath = model_name + '_weights.hdf5'

# serialize model to JSON
model_json = model.to_json()
with open(model_filepath, 'w') as json_file:
    json_file.write(model_json) 
    
# define the model checkpoint and Tensorboard callbacks
checkpoint = ModelCheckpoint(weights_filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
tensorboard = TensorBoard(os.path.join('logs', model_name))
callbacks_list = [checkpoint, tensorboard]

# fit model
history = model.fit(train_gen, 
          epochs = 25, 
          batch_size = 32, 
          steps_per_epoch=train_steps,
          validation_data=val_gen,
          validation_steps=val_steps,
          callbacks=callbacks_list)

val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

Epoch 1/25
4500/4500 [==============================] - 243s 52ms/step - loss: 0.8488 - accuracy: 0.5661 - val_loss: 0.6394 - val_accuracy: 0.6924

Epoch 00001: val_loss improved from inf to 0.63935, saving model to ResNet_Stage12345_s_weights.hdf5
Epoch 2/25
4500/4500 [==============================] - 233s 52ms/step - loss: 0.6558 - accuracy: 0.6860 - val_loss: 0.5773 - val_accuracy: 0.7256

Epoch 00002: val_loss improved from 0.63935 to 0.57734, saving model to ResNet_Stage12345_s_weights.hdf5
Epoch 3/25
4500/4500 [==============================] - 234s 52ms/step - loss: 0.6045 - accuracy: 0.7119 - val_loss: 0.5477 - val_accuracy: 0.7433

Epoch 00003: val_loss improved from 0.57734 to 0.54774, saving model to ResNet_Stage12345_s_weights.hdf5
Epoch 4/25
4500/4500 [==============================] - 238s 53ms/step - loss: 0.5760 - accuracy: 0.7279 - val_loss: 0.5351 - val_accuracy: 0.7456

Epoch 00004: val_loss improved from 0.54774 to 0.53506, saving model to ResNet_Stage12345_s_weigh

In [15]:
#hypermodel = tuner.hypermodel.build(best_hps)

# Retrain the model
#hypermodel.fit(img_train, label_train, epochs=best_epoch, validation_split=0.2)

In [16]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # or any {'0', '1', '2'}   
import tensorflow as tf
import numpy as np
import glob
import pandas as pd
from matplotlib.pyplot import imread
from tensorflow.keras.models import model_from_json

#Change these variables to point at the locations and names of the test dataset and your models.
TEST_PATH = r'data/test/'
MODEL_FILEPATH = 'ResNet_Stage12345_s.json' 
MODEL_WEIGHTS_FILEPATH = 'ResNet_Stage12345_s_weights.hdf5'

# load model and model weights
json_file = open(MODEL_FILEPATH, 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)

# load weights into new model
model.load_weights(MODEL_WEIGHTS_FILEPATH)

# open the test set in batches (as it is a very big dataset) and make predictions
test_files = glob.glob(TEST_PATH + '*')
print(test_files)
submission = pd.DataFrame()

file_batch = 5000
max_idx = len(test_files)

for idx in range(0, max_idx, file_batch):

    print('Indexes: %i - %i'%(idx, idx+file_batch))

    test_df = pd.DataFrame({'path': test_files[idx:idx+file_batch]})


    # get the image id 
    test_df['id'] = test_df.path.map(lambda x: x.split(os.sep)[-1].split('.')[0])
    test_df['image'] = test_df['path'].map(imread)
    
    
    K_test = np.stack(test_df['image'].values)
    
    # apply the same preprocessing as during draining
    K_test = K_test.astype('float')/255.0
    
    predictions = model.predict(K_test)
    
    test_df['label'] = predictions
    submission = pd.concat([submission, test_df[['id', 'label']]])


# save your submission
submission.head()
submission.to_csv('submission_ResNet_Stage12345_s.csv', index = False, header = True)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Indexes: 5000 - 10000
Indexes: 10000 - 15000
Indexes: 15000 - 20000
Indexes: 20000 - 25000
Indexes: 25000 - 30000
Indexes: 30000 - 35000
Indexes: 35000 - 40000
Indexes: 40000 - 45000
Indexes: 45000 - 50000
Indexes: 50000 - 55000
Indexes: 55000 - 60000
